In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from tensorflow.keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [4]:
df = pd.read_csv("WELFake_Dataset.csv")

In [5]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
df.isna().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [7]:
if "Unnamed: 0" in df.columns:
    df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
df.fillna("")
headline = df["title"]
news = df["text"]
df["tot_news"] = df["title"].astype("str") + " " + df["text"].astype("str")
tot_news = df["tot_news"]
labels = df["label"]

In [9]:
fake_news = news[labels == 0]
real_news = news[labels == 1]

In [10]:
X = tot_news

Y = labels.values
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=73939133)

In [20]:
max_features = 10000
maxlen = 300
batch_size = 256
epochs = 2
embed_size = 100
# embed_size = 300

In [21]:
tokenizer = text.Tokenizer(num_words=max_features,  lower=None)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)
tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

AttributeError: 'numpy.ndarray' object has no attribute 'translate'

In [22]:
EMBEDDING_FILE = "glove.840B.300d.txt"

In [18]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 962: character maps to <undefined>

In [19]:
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

NameError: name 'embeddings_index' is not defined

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=2, 
    verbose=True,
    factor=0.6, 
    min_lr=0.00001
)

In [ ]:
#Defining Neural Network
model1 = Sequential()
#Non-trainable embeddidng layer
model1.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model1.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model1.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model1.add(Dense(units = 32 , activation = 'relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

In [ ]:
history = model1.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

In [ ]:
print("Accuracy of the model on Training Data is - " , model1.evaluate(x_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model1.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
pred = model1.predict(x_test)

In [ ]:
y_pred = np.where(pred >0.5, 1, 0)

In [15]:
print(classification_report(y_test, y_pred, target_names = ['Fake','Not Fake']))

NameError: name 'y_pred' is not defined

In [ ]:

confmat = confusion_matrix(y_pred,y_test)

In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix, 
    accuracy_score,
    ConfusionMatrixDisplay
)

In [ ]:
print(f"Model 1 confusion matrix :")
# y_test # array([1, 0, 1, ..., 1, 1, 1])
disp = ConfusionMatrixDisplay(confmat)
disp.plot()
plt.savefig('confmat_model4.png', dpi=300)
plt.show()


In [ ]:
model2 = Sequential()
model2.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model2.add(LSTM(100))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
print(model2.summary())

In [ ]:
history2 = model2.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs )

In [ ]:
def best_threshold_value(thresholds:list,X_test):
    accuracies = []
    for thresh in thresholds:
        ypred =model2.predict(X_test)
        ypred = np.where(ypred> thresh,1,0)
        accuracies.append(accuracy_score(y_test,ypred))
    return pd.DataFrame({
        'Threshold': thresholds,
        'Accuracy' : accuracies
    })

In [ ]:
best_threshold_value([0.4,0.5,0.6,0.7,0.8,0.9], x_test)

In [ ]:
y_pred = model2.predict(x_test)
y_pred = np.where(y_pred >0.4, 1, 0)

In [ ]:
print(classification_report(y_test, y_pred, target_names = ['Fake','Not Fake']))

In [ ]:
confmat2 = confusion_matrix(y_pred,y_test)

In [ ]:
print(f"Model 2 confusion matrix :")
# y_test # array([1, 0, 1, ..., 1, 1, 1])
disp = ConfusionMatrixDisplay(confmat2)
disp.plot()
plt.savefig('confmat_model4.png', dpi=300)
plt.show()

In [ ]:
model2.summary()

In [ ]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
model3=Sequential()
model3.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

In [ ]:
history3 = model3.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs )

In [ ]:
y_pred2 = model3.predict(x_test)
y_pred2 = np.where(y_pred2 >0.4, 1, 0)

In [ ]:
print(classification_report(y_test, y_pred2, target_names = ['Fake','Not Fake']))

In [ ]:
confmat3 = confusion_matrix(y_pred2,y_test)
print(f"Model 3 confusion matrix :")
disp = ConfusionMatrixDisplay(confmat3)
disp.plot()
plt.savefig('confmat_model4.png', dpi=300)
plt.show()

In [ ]:
model3.summary()

In [ ]:
model2.save("./simplelstm.keras")

In [ ]:
model3.save("./bi_lstm.keras")